##selenium을 활용하여 월스트리트저널 페이지에서 뉴스 크롤링하기

In [3]:
#selenium setting
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

##각자 크롬 버전에 맞는 chromedriver.exe가 있어야함 (코드 파일과 chromedriver.exe 같은 폴더 내)

In [4]:
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.3")
#options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/323.434 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/123.45')
#위와 같은 양식으로 진행 (예시에 나와있는 숫자는 임의로 변경함. 각자의 user-agent입력) : user agent는 구글에서 what is my user agent검색

In [5]:
driver = webdriver.Chrome(options=options)#option 설정

In [6]:
#wallstreet_filter열기
#(WSJ의 search란에서 기간과 검색 단어 설정 후 사이트 설정)
url = f'https://www.wsj.com/search?query=tech&isToggleOn=true&operator=OR&sort=date-desc&duration=1y&startDate=2023%2F01%2F01&endDate=2023%2F07%2F24&source=wsjaudio%2Cwsjie%2Cautowire%2Clivecoverage'

In [7]:
driver.get(url) #url_open


##WSJ의 경우 구독자들에게만 기사를 제공하기 때문에, 여기서 로그인 과정을 무조건 진행해줘야함!

##sign in을 통하여 로그인 진행하기(구독 완료 했어야 함)
로그인을 완료한 후 아래코드 실행하기

In [8]:
#selenium예외처리 설정
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import TimeoutException

In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

with open('WSJ_IT.csv', 'w', encoding='utf-8', newline='') as f: #WSJ_IT.CSV에 파일 저장
    writer = csv.writer(f)
    writer.writerow(['title', 'content', 'date', 'href'])#행 입력
    for year in range(2023,2024): #1년단위로 추출 진행
        for page in range(1, 375): #마지막 페이지 번호 확인 후 +1
            url = f'https://www.wsj.com/search?query=tech&isToggleOn=true&operator=OR&sort=date-desc&duration=1y&startDate={year}%2F01%2F01&endDate={year}%2F07%2F24&source=wsjaudio%2Cwsjie%2Cautowire%2Clivecoverage&page={page}'
            driver.get(url)

            #아래는 예외처리 과정
            try: 
                boxes = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "main"))).find_elements(By.TAG_NAME, "article")
                #boxes: presence_of_element_located, ID "main", Tag_Name "article"이 나올때 까지 10초 기다려라
            except TimeoutException:
                continue  # 10초 안에 로드되지 않으면 다음 페이지로 넘어가기
            
            if len(boxes) == 0: #찾아온게 없음_길이가 0이면
                break
            for box in boxes: #boxes라는 리스트 안의 각 box에서 기사의 제목을 찾은 후 class찾고 a 태그 찾기
                try:
                    headline = WebDriverWait(box, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'WSJTheme--headline--7VCzo7Ay'))).find_element(By.TAG_NAME, 'a') #headline찾기
                    href = headline.get_attribute('href')#headline에서 href라는 url을 가지고 있는 값 저장
                    title = headline.text
                except (StaleElementReferenceException, TimeoutException): #코드를 돌리다보니 이러한 오류가 많이 발생하여, 예외처리를 진행하였습니다.
                    continue

                print(f'url: {url} // title: {title}') #코드가 잘 진행되는지 확인하기 위하여 print문 진행
                script = f"window.open('{href}');" #본문 script 사이트 저장
                driver.execute_script(script) #본문 script 열기

                try: #새로운 열린 창(본문 script) 드라이버의 포커스를 이동 / 열린 창들을 리스트로 설정
                    driver.switch_to.window(driver.window_handles[1])#있으면 새로운 본문[1] 창으로 이동
                except NoSuchWindowException:
                    driver.switch_to.window(driver.window_handles[0])#없으면 이전에 headline만 있는 창[0]으로 이동
                    continue #다음 box로 이동

                try:
                    date = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "css-10158ns-TimeTag"))) #data 클래스 발견
                    date = date.text #date class를 가진 date에서 text만 추출
                except (NoSuchElementException, TimeoutException):
                    date = None

                try:
                    content = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ef4qpkp0"))) #content 클래스 발견
                    content = content.text#contet class를 가진 contetn에서 text만 추출
                except (NoSuchElementException, TimeoutException):
                    content = None
                
                writer.writerow([title, content, date, href]) #한 행에 지금까지 받은 데이터 저장

                driver.close() #[1]로 되어있는 본문있는 새로운 창 닫기

                driver.switch_to.window(driver.window_handles[0]) #필터링 되어있는 검색 창으로 이동


url: https://www.wsj.com/search?query=tech&isToggleOn=true&operator=OR&sort=date-desc&duration=1y&startDate=2023%2F01%2F01&endDate=2023%2F07%2F24&source=wsjaudio%2Cwsjie%2Cautowire%2Clivecoverage&page=1 // title: Elon Musk’s Pivot to X Draws Strong Opinions Across Twitter
url: https://www.wsj.com/search?query=tech&isToggleOn=true&operator=OR&sort=date-desc&duration=1y&startDate=2023%2F01%2F01&endDate=2023%2F07%2F24&source=wsjaudio%2Cwsjie%2Cautowire%2Clivecoverage&page=1 // title: Tech Up Slightly Ahead of Key Earnings -- Tech Roundup


KeyboardInterrupt: 